# Exploratory data analysis of the Berlin dataset

Dataset can be found on https://www.kaggle.com/datasets/mathisjander/real-estate-listings-berlin-de-april-2023.

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [5]:
# Colorblind-friendly colors
colors = sns.color_palette("colorblind", 10)
sns.set_palette("colorblind")

### Data loading and quick exploration

In [6]:
data = pd.read_csv('../data/berlin_houses_apr2023.csv')

# Show first rows
data.head()

,url,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
0,https://www.immowelt.de/expose/29fhx5b,Öl,Zentralheizung,249000.0,64.00,3.0,3.57,13505.0,1971.0,2.0,3890.625000
1,https://www.immowelt.de/expose/29hzk5u,Gas,"Fußbodenheizung, offener",1295000.0,136.06,4.0,3.57,10405.0,1900.0,5.0,9517.859768
2,https://www.immowelt.de/expose/292m95u,Gas,Etagenheizung,770000.0,120.00,4.0,3.57,12161.0,1900.0,5.0,6416.666667
3,https://www.immowelt.de/expose/29sh55u,Gas,Zentralheizung,349000.0,158.00,5.0,3.57,12103.0,1956.0,1.0,2208.860759
4,https://www.immowelt.de/expose/29r5y5t,Gas,Etagenheizung,494990.0,126.00,4.0,3.57,12157.0,1909.0,3.0,3928.492063


In [7]:
# Display key properties
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4942 entries, 0 to 4941
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   url                4942 non-null   object 
 1   energy             4942 non-null   object 
 2   heating            4942 non-null   object 
 3   price              4942 non-null   float64
 4   area               4942 non-null   float64
 5   rooms              4942 non-null   float64
 6   fee                4942 non-null   float64
 7   zipcode            4942 non-null   float64
 8   construction_year  4942 non-null   float64
 9   level              4942 non-null   float64
 10  price_per_area     4942 non-null   float64
dtypes: float64(8), object(3)
memory usage: 424.8+ KB


From the above and the description of the dataset on Kaggle, we have :
url
link to the listing


energy
energy source type


heating
heating type


price
price of listing


area
area in m2


rooms
number of rooms


fee
agent fee


zipcode
ZIP code in Berlin


construction_year
Year of construction


level
Floor level of listing

In [8]:
# Check unique values for each feature
print(f'Number of different values (on {len(data)} samples): \n\n{data.nunique()}')

Number of different values (on 4942 samples): 

url                  4942
energy                 20
heating                23
price                1430
area                 2119
rooms                  21
fee                    68
zipcode               190
construction_year     155
level                  20
price_per_area       3601
dtype: int64


In [9]:
# Check missing entries
print(f'Number of missing values : \n\n{data.isnull().sum()}')

Number of missing values : 

url                  0
energy               0
heating              0
price                0
area                 0
rooms                0
fee                  0
zipcode              0
construction_year    0
level                0
price_per_area       0
dtype: int64


In [10]:
# Check duplicates
print(f'Number of duplicated rows : {data.duplicated().sum()}')

Number of duplicated rows : 0


In [11]:
data.describe()

,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
count,4.942000e+03,4942.000000,4942.000000,4942.000000,4942.000000,4942.000000,4942.000000,4942.000000
mean,5.654009e+05,85.067321,2.805544,5.891574,11954.461554,1944.344193,3.401255,6401.269284
std,6.107572e+05,58.315922,1.582906,121.004862,1628.058346,143.609751,29.137154,3183.956274
min,3.495000e+04,13.000000,1.000000,-3.500000,1099.000000,1.000000,1.000000,956.521739
25%,2.590000e+05,54.070000,2.000000,3.570000,10589.000000,1910.000000,1.000000,4537.537879
50%,3.899000e+05,72.740000,3.000000,3.570000,12161.000000,1956.000000,1.000000,5843.852267
75%,6.690000e+05,101.000000,3.000000,3.570000,13088.000000,1994.000000,3.000000,8063.468346
max,1.590000e+07,970.000000,26.000000,7996.000000,77933.000000,2026.000000,2023.000000,87739.070663


#### Summary : 